1) [Задача классификации изображений](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html?highlight=mnist). Повторить тренировку модели (train) и запустить классификацию изображений (inference).

In [171]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [172]:
# dataset

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 2
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [173]:
# define a CNN

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def train(self, trainloader, num_epochs=2, learning_rate=0.001, momentum=0.9, device=torch.device('cpu')):
        self.to(device)
        # define a Loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(self.parameters(), lr=learning_rate, momentum=momentum)

        # train the network
        for epoch in range(num_epochs):  # loop over the dataset multiple times

            running_loss = 0.0
            for i, data in enumerate(trainloader, 0):
                # get the inputs; data is a list of [inputs, labels]
                inputs, labels = data[0].to(device), data[1].to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = self(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                # print statistics
                running_loss += loss.item()
                if i % 2000 == 1999:    # print every 2000 mini-batches
                    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                    running_loss = 0.0

        print('Finished Training')


In [174]:
net = Net()

# По возможности обучить на GPU
cuda = torch.cuda.is_available()
if cuda:
    device = torch.device('cuda:0')
    net.to(device)
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)


cuda:0


In [175]:
net.train(trainloader, device=device, num_epochs=8, learning_rate=0.0007)

[1,  2000] loss: 2.247
[1,  4000] loss: 1.983
[1,  6000] loss: 1.825
[1,  8000] loss: 1.724
[1, 10000] loss: 1.644
[1, 12000] loss: 1.617
[1, 14000] loss: 1.576
[1, 16000] loss: 1.546
[1, 18000] loss: 1.488
[1, 20000] loss: 1.462
[1, 22000] loss: 1.468
[1, 24000] loss: 1.433
[2,  2000] loss: 1.395
[2,  4000] loss: 1.389
[2,  6000] loss: 1.373
[2,  8000] loss: 1.350
[2, 10000] loss: 1.336
[2, 12000] loss: 1.323
[2, 14000] loss: 1.327
[2, 16000] loss: 1.289
[2, 18000] loss: 1.321
[2, 20000] loss: 1.281
[2, 22000] loss: 1.288
[2, 24000] loss: 1.267
[3,  2000] loss: 1.175
[3,  4000] loss: 1.195
[3,  6000] loss: 1.175
[3,  8000] loss: 1.210
[3, 10000] loss: 1.197
[3, 12000] loss: 1.181
[3, 14000] loss: 1.180
[3, 16000] loss: 1.214
[3, 18000] loss: 1.208
[3, 20000] loss: 1.185
[3, 22000] loss: 1.189
[3, 24000] loss: 1.144
[4,  2000] loss: 1.066
[4,  4000] loss: 1.110
[4,  6000] loss: 1.061
[4,  8000] loss: 1.079
[4, 10000] loss: 1.140
[4, 12000] loss: 1.098
[4, 14000] loss: 1.115
[4, 16000] 

In [176]:
# test the network on the test data

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 59 %


In [177]:
# save trained model
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)